<h4>(1 - Recommend). Create and save new notebook(s) to enable automatic re-mounting of Drive storage. Copy and paste the below code into a <a href="http://colab.research.google.com/#create=true" target="_parent">new notebook</a> in Colab and follow the directions in the <a href="../#how-to-use" target="_parent">How To Use</a> section of the <em>README</em></h4><h4>(2). Open the pre-formed version in Colab (requires manual authorization each time a notebook is opened) <a href="https://colab.research.google.com/github/tdulcet/Distributed-Computing-Scripts/blob/master/google-colab/GoogleColabCPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h4>

In [ ]:
#@title # **📔 Colab CPU Notebook**#{ vertical-output: true, form-width: "96%", display-mode: "form" }

#@markdown #### ↖️ Click the ▶️ button after deciding on the options below.
#@markdown #### 💡 Keep each notebook **open** in the browser to prevent disconnection. 📌 [the tab](https://support.mozilla.org/en-US/kb/pinned-tabs-keep-favorite-websites-open) or move them to a dedicated window for easy access.
#@markdown #### ℹ️ This notebook uses our Prime95 [Bash install script](https://github.com/tdulcet/Distributed-Computing-Scripts#organizations).
#@markdown #### 📜 Please see the [documentation](https://github.com/tdulcet/Distributed-Computing-Scripts/tree/master/google-colab) for more information and to ❤️ support us.
#@markdown #### 🤷 Optionally, create a GIMPS/PrimeNet account [here](https://www.mersenne.org/update/) and [join](https://www.mersenne.org/jteam/) the “Portland State University” team!

prime_ID = 'Default' #@param ['Default'] {allow-input: true}
computer_name = 'Default' #@param ['Default'] {allow-input: true}
type_of_work = '150 - First time PRP tests' #@param ['0 - Whatever makes the most sense', '1 - Trial factoring to low limits', '2 - Trial factoring', '4 - P-1 factoring', '5 - ECM for first factors of Mersenne numbers', '6 - ECM on Fermat numbers', '8 - ECM on Mersenne cofactors', '100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests', '150 - First time PRP tests', '151 - Double-check PRP tests', '152 - World record PRP tests', '153 - 100 million digit PRP tests', '154 - First time PRP tests that need P-1 factoring', '155 - Double-check tests using PRP with proof', '160 - First time PRP on Mersenne cofactors', '161 - Double-check PRP on Mersenne cofactors']
prp_proof_power = '5' #@param ['5', '6', '7', '8', '9', '10', '11', '12']
#@markdown Every lower power halves Drive storage requirements for PRP tests, but doubles the certification cost.
proof_certification_work = True #@param {type:"boolean"}
computer_number = 'Default (1)' #@param ['Default (1)', '2', '3', '4']{allow-input: true}
local_time = 'Pacific' #@param ['Pacific', 'Mountain', 'Central', 'Eastern', 'Alaska', 'Hawaii']
debug = 'False' #@param ['False', 'CPU (Prime95)']

#@markdown #### 🐛 The *debug* option outputs CPU (Prime95/MPrime) progress and status, then exits.

import os

class StopExecution(Exception):
  def _render_traceback_(self):
      pass

def load_drive():
  '''Load & cd into gdrive for persistent data'''
  if os.path.exists("/content/drive/My Drive"): # create your own notebook with our code
    %cd "/content/drive/My Drive"
  else: # use our notebook
    print('Warning: Google Drive is not mounted')
    print('If you were not expecting this, on the far left click the folder icon, the "Mount Drive" folder button, select "CONNECT TO GOOGLE DRIVE" ')
    print('and then re-execute this cell.')
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd "/content/gdrive/My Drive/"
  if "My Drive/GIMPS" in os.getcwd(): # don't create a subfolder in GIMPS/
    return
  !mkdir -p GIMPS
  %cd "GIMPS"

def run():
  '''Run MPrime'''
  print('\nStarting Prime95\n')
  !cd mprime_cpu && chmod 777 mprime && ./mprime -A$computer_number -d | tee -a "{'cpu' + computer_number + '.out'}"

def debug_exit():
  '''Output of Prime95 CPU outputs'''
  if debug == 'CPU (Prime95)' and os.path.exists('mprime_cpu/cpu' + computer_number + '.out'):
    print("\nCPU (Prime95) output:\n")
    !tail -n 100 "{'mprime_cpu/cpu' + computer_number + '.out'}" # view MPrime progress
    !cd mprime_cpu && chmod 777 mprime && ./mprime -s -A$computer_number
    print()
  else:
    print(f'No `{debug}` output file found for debug option and computer number `{computer_number}`.\n')

def install():
  '''Download/Install/Configure Prime95'''
  !wget https://github.com/tdulcet/Distributed-Computing-Scripts/archive/master.zip -nv -O master.zip
  !unzip -o master.zip
  print("Downloading and setting up Prime95\n")
  !cp Distributed-Computing-Scripts-master/{mprime2.sh,mprime2.exp,idletime.sh} .
  !sed -i 's/"mprime"/"mprime_cpu"/' mprime2.sh # Name the folder specific to the runtime type
  !sed -i '/^\.\/mprime / s/^/# /' mprime2.sh # Do not start Prime95
  !sed -i '/^nohup / s/^/# /' mprime2.sh # Do not start Prime95
  !sed -i '/^crontab / s/^/# /' mprime2.sh # Do not create a cronjob
  !sed -i '/^expect {/a \\t"Max emergency memory in GB/worker (*):" { sleep 1; send -- "3\\r"; exp_continue }\n\t"Get occasional proof certification work (*):" { sleep 1; send -- "PROOF_CERTIFICATION_WORK\\r"; exp_continue }' mprime2.exp
  !sed -i 's/PROOF_CERTIFICATION_WORK/'$proof_certification_work'/' mprime2.exp
  ![[ -d 'mprime_cpu' ]] && cd mprime_cpu && chmod 777 mprime
  !bash -- mprime2.sh $computer_number $prime_ID $computer_name $type_of_work # Run script
  file = '"' + f'mprime_cpu/prim{int(computer_number):04d}.txt' + '"'
  !echo 'FixedHardwareUID=1' > temp.txt
  !echo 'ProofPower={prp_proof_power}' >> temp.txt
  !cat $file >> temp.txt
  !mv temp.txt $file
  !echo 'PreallocateDisk=0' >> '"' + f'mprime_cpu/loca{int(computer_number):04d}.txt' + '"'
  run()

!wget https://raw.github.com/tdulcet/Linux-System-Information/master/info.sh -qO - | bash -s # Check System Info
!python3 -V
print()
load_drive()

# set local time
!rm -f /etc/localtime
!ln -s {'/usr/share/zoneinfo/US/' + local_time} /etc/localtime

# use/cleanup input from user
prime_ID = 'psu' if prime_ID.lower() == 'default' else prime_ID
computer_name = "" if computer_name.lower() == 'default' else computer_name
type_of_work = type_of_work.split("-")[0].rstrip()
proof_certification_work = "y" if proof_certification_work else "n"
computer_number = "1" if computer_number.lower() == 'default (1)' else computer_number.strip()
debug = False if debug == 'False' else debug

# Add quotes to string args so script can parse spaces/special characters
prime_ID = '"' + prime_ID + '"'
computer_name = '"' + computer_name + '"'


if debug:
  debug_exit()
  raise StopExecution

elif not computer_number.isdigit() or int(computer_number) < 0:
  print("ERROR: Computer number must be a number")
  raise StopExecution

elif os.path.exists(f'mprime_cpu/work{int(computer_number):04d}.txt'):
  !cd mprime_cpu && echo -e "$(date)\t$(sed -n 's/^model name[[:space:]]*: *//p' /proc/cpuinfo | uniq)  $(sed -n 's/^model[[:space:]]*: *//p' /proc/cpuinfo | uniq)" >> cpus.txt
  print('\nPrevious CPU counts')
  !cd mprime_cpu; awk -F'\t' '{ print $2 }' cpus.txt | sort | uniq -c | sort -nr
  run()

else:
  install()

print("Gracefully exiting...")